In [ ]:
from src import Color
from src.material.material.phong_material import PhongMaterial
from src import PointLight
from src import Resolution
from src import Camera
from src import Scene
from src.geometry.object_geometry import ObjectGeometry
from src.geometry.geometry_hit import GeometryHit
from src.geometry.ray import Ray
from src.math import Vertex, Vector
from dataclasses import dataclass
from src.render import LinearRayCaster
from src.render.loops.progress import PreviewConfig, ProgressDisplay
from src.render.render_config import RenderConfig
import numpy as np

# You can create your own geometry objects by implementing the Hittable interface.
- Here is an example of a Torus (doughnut shape) implementation and its usage in a simple scene.
- hittable interface requires implementing `intersect`, `random_point`, and `normal_at` methods.

In [ ]:
@dataclass
class MySphere(ObjectGeometry):
    """
    Sphere in 3D space defined by center, radius, and color.
    """
    center: Vertex
    radius: float

    def normal_at(self, point: Vertex) -> Vector:
        normal = (point - self.center) / self.radius
        return normal

    def intersect(self, ray: Ray, t_min=0.001, t_max=float('inf')) -> GeometryHit | None:
        oc = ray.origin - self.center

        # Quadratic coefficients
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius

        discriminant = b * b - 4 * a * c

        if discriminant < 0: # no intersection
            return None

        sqrt_disc = np.sqrt(discriminant)

        # Find the nearest root
        root = (-b - sqrt_disc) / (2.0 * a)
        if root < t_min or root > t_max:
            root = (-b + sqrt_disc) / (2.0 * a)
            # Point out of bounds
            if root < t_min or root > t_max:
                return None

        hit_point = ray.point_at(root)

        normal = self.normal_at(hit_point)
        # Ensure the normal is facing against the ray
        if ray.direction.dot(normal) > 0.0:
            normal = -normal

        return GeometryHit(dist=root, point=hit_point, normal=normal, front_face=True, geometry_id=-1)

    def random_point(self) -> Vertex:
        pass

In [ ]:
from src.scene.primitive import Object

glossy_red = PhongMaterial(
    name="glossy_red",
    base_color=Color.custom_rgb(255, 0, 0),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=30,
)


point_light = PointLight(position=Vertex(5, 5, 0), intensity=2000.0, falloff=0.01)

camera = Camera(
    fov = 30,
    aspect_ratio = Resolution.R360p.aspect_ratio,
    origin = Vertex(0, 4, 0),
    direction = Vector(0, -0.7, -1),
)

scene = Scene(
    camera = camera,
    primitives=[
        Object(
            MySphere(Vertex(0.0, 0.0, -6), 1.0),
            glossy_red,
        ),
    ],
    lights = [point_light],
    skybox_path ="../skybox/shanghai_4k.hdr"
)

render_configuration = RenderConfig(
    resolution=Resolution.R480p,
    samples_per_pixel=1,
    max_depth=2,
)

preview_configuration = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

my_ray_tracer = LinearRayCaster(scene=scene, render_config=render_configuration, preview_config=preview_configuration)
png = my_ray_tracer.render("../images/custom_object.png")